In [ ]:
#Do imports here please
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

!pip install influxdb
import influxdb

!pip3 install iso8601
import iso8601

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 1.8 MB/s eta 0:00:00


In [ ]:
#Taken from CPS1 Assignment A of group member Aidan, who is in both classes
HOST = 'influx.linklab.virginia.edu'
PORT = 443
USERNAME = 'cps1f23'
PASSWORD = 'phah7goohohng5ooL9mae1quohpei1Ahsh1uGing'
DATABASE = 'gateway-generic'

client = influxdb.InfluxDBClient(HOST, PORT, USERNAME, PASSWORD, DATABASE, ssl=True, verify_ssl=True)

def gatherData(firstMeasure = "Temperature_°C", secondMeasure = "Illumination_lx"):
  firstData = client.query(f"SELECT MEAN(value) FROM \"{firstMeasure}\" WHERE time >= '2024-05-01T00:00:00Z' AND time <= '2024-05-31T00:00:00Z' GROUP BY time(10m),device_id,location_specific")
  secondData = client.query(f"SELECT MEAN(value) FROM \"{secondMeasure}\" WHERE time >= '2024-05-01T00:00:00Z' AND time <= '2024-05-31T00:00:00Z' GROUP BY time(10m),device_id,location_specific")

  extracted_data = []
  n = 0
  for i in firstData:
    for j in i:
      data = {'location_specific': firstData.keys()[n][1]['location_specific'], 'device_id': firstData.keys()[n][1]['device_id'],'time': j['time'], firstMeasure: j['mean']}
      extracted_data.append(data)

    n += 1

  firstDF = pd.DataFrame(extracted_data)

  extracted_data = []
  n = 0
  for i in secondData:
    for j in i:
      data = {'location_specific': secondData.keys()[n][1]['location_specific'], 'device_id': secondData.keys()[n][1]['device_id'],'time': j['time'], secondMeasure: j['mean']}
      extracted_data.append(data)

    n += 1

  secondDF = pd.DataFrame(extracted_data)

  return pd.merge(firstDF, secondDF, on=['time','location_specific'], how='inner').dropna()

# Original Data Gathering

In [ ]:
#This takes a very long time to run, probably don't. Aidan has a csv with the output saved on google drive, check the discord for it
raw_data = gatherData(firstMeasure = "co2_ppm", secondMeasure = "voltage_v")

KeyboardInterrupt: 

In [ ]:
raw_data.to_csv('data.csv')

In [ ]:
result = client.query('SELECT mean(value) FROM "voltage_v" WHERE time > now()-6h GROUP BY time(1m), device_id, location_specific;')
for k,v in result.items():
  print(k)
  for d in v:
    print(d)

# Basic Graphs (M2)

In [ ]:
#get data for 211 olson for the last day for both CO2 and voltage
x1 = []
y1 = []
result = client.query('SELECT mean(value) FROM "co2_ppm" WHERE time > now() - 300d AND time < now() - 293d AND location_specific = \'277 Olsson\' GROUP BY time(15m);')
for k,v in result.items():
  for d in v:
    x1.append(iso8601.parse_date(d['time']))
    y1.append(d['mean'])

x2 = []
y2 = []
result = client.query('SELECT mean(value) FROM "power_w" WHERE time > now() - 300d AND time < now() - 293d AND location_specific = \'277 Olsson\' GROUP BY time(15m);')
for k,v in result.items():
  for d in v:
    x2.append(iso8601.parse_date(d['time']))
    y2.append(d['mean'])


#with reference to https://matplotlib.org/3.4.3/gallery/ticks_and_spines/multiple_yaxis_with_spines.html
#plotting the two


fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)

twin1 = ax.twinx()


p1, = ax.plot(x1, y1, "b-", label="CO2")
p2, = twin1.plot(x2, y2, "r-", label="Power")

#ax.set_xlim(0, 2)
#ax.set_ylim(0, 2)
#twin1.set_ylim(0, 4)

ax.set_xlabel("Time")
ax.set_ylabel("CO2 (ppm)")
twin1.set_ylabel("Power (W)")

ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p2.get_color())

tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
ax.tick_params(axis='x', **tkw)

ax.legend(handles=[p1, p2])

plt.show()

NameError: name 'client' is not defined